# Development of polymer building, including monomer and bonding spec

In [ ]:
# Supressing annoying warnings (!must be done first!)
import warnings
warnings.catch_warnings(record=True)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

import logging
from polymerist.genutils.logutils.IOHandlers import LOG_FORMATTER

logging.basicConfig(
    level=logging.INFO,
    format =LOG_FORMATTER._fmt,
    datefmt=LOG_FORMATTER.datefmt,
    force=True
)
LOGGER = logging.getLogger(__name__)

from rdkit import Chem
from rdkit.Chem import RWMol, BondType

from polymerist import rdutils
from polymerist.rdutils import rdkdraw
from polymerist.rdutils.rdtypes import RDAtom, RDBond, RDMol
from polymerist.rdutils.labeling import atomwise, molwise
from polymerist.rdutils.labeling.molwise import assign_ordered_atom_map_nums
from polymerist.rdutils.amalgamation import bonding, portlib

from polymerist.genutils.iteration import iter_len
from polymerist.genutils.decorators.functional import optional_in_place

# Debugging NGLView issues

In [1]:
import nglview as nv
from openff.toolkit import Molecule

In [2]:
import nglview as nv

nv.demo()

NGLWidget()

In [3]:
from openff.toolkit import Molecule

m = Molecule.from_smiles('CC=O')
m.generate_conformers(n_conformers=1)
# m.visualize(backend='rdkit')
m.visualize(backend='nglview')

NGLWidget()

# Port-binding rules

## Test structures

In [ ]:
# some "normal" molecules for testing
H  = Chem.MolFromSmarts('[#1]-[1#0]')
OH = Chem.MolFromSmarts('[#1]-[O]-[2#0]')
METHYL = Chem.MolFromSmarts('[#6](-[2#0])(-[2#0])(-[3#0])(-[1#0])')
CARBONYL = Chem.MolFromSmarts('[#6]-[#6](=[#8])-[1#0]')
WITTIG = Chem.MolFromSmarts('[#6](=[1#0])(-[2#0])(-[#1])')
WITTIG_DUAL = Chem.MolFromSmarts('[#6](=[1#0])(-[#7](-[#1])(-[#1]))(-[#1])')

TEST_MOLS_NORMAL = (
    H,
    OH,
    METHYL,
    WITTIG,
    WITTIG_DUAL,
    CARBONYL
)

# Pathological examples for debug
DOUBLE_MID = Chem.MolFromSmarts('[#6](-[1#0])(-[#1])=[#0]-[#6](-[#1])(-[#1])(-[2#0])')
NEUTRONIUM = Chem.MolFromSmarts('[#0]-[#0]')
GHOST_WATER = Chem.MolFromSmarts('[#1]-[#0]-[#1]')

TEST_MOLS_PATHO = (
    DOUBLE_MID,
    NEUTRONIUM,
    GHOST_WATER
)

# COMBINE FOR UNIVERSAL TESTING
TEST_MOLS = (
    *TEST_MOLS_NORMAL,
    *TEST_MOLS_PATHO
)

In [ ]:
rdkdraw.disable_substruct_highlights()

for mol in TEST_MOLS:
    assign_ordered_atom_map_nums(mol, in_place=True)
    print(portlib.get_num_ports(mol))
    display(mol)

## Testing bond dissolution and splicing 

In [ ]:
a1 = 2 - 1 # indices of atoms to target
a2 = 3 - 1 # indices of atoms to target
flavor_pair = (1, 3) # pair of port flavors to interface together

display(CARBONYL)
frags = bonding.dissolve_bond(Chem.RWMol(CARBONYL), a1, a2, new_port_flavor=3)
MET, OXY = Chem.GetMolFrags(frags, asMols=True, sanitizeFrags=True)

display(frags)
display(MET)
display(OXY)

portlib.Port.bondable_flavors.insert(flavor_pair)
bondable_pair = portlib.get_first_bondable_port_pair(frags, a1, a2, flavor_pair=flavor_pair)
prod1 = bonding.splice_atoms(frags, 1, 2, flavor_pair=(1,3), in_place=False)
display(prod1)

In [ ]:
TEST = bonding.combined_rdmol(METHYL, OXY)
display(TEST)

order2 = portlib.max_bondable_order_between_atoms(TEST, 0, 5, target_flavor=3)
prod2 = bonding.splice_atoms(TEST, 0, 5, flavor_pair=(1,3))

print(order2)
display(prod2)

## Testing port saturation

### Testing single bonds

In [ ]:
H_GENERIC = Chem.MolFromSmarts('[#1]-[#0]')
OH_GENERIC = Chem.MolFromSmarts('[#1]-[O]-[4#0]')
METHYL_GENERIC = Chem.MolFromSmarts('[#6](-[#0])(-[#0])(-[#0])(-[#0])')

display(H_GENERIC)
display(OH_GENERIC)
display(METHYL_GENERIC)

In [ ]:
bonding.saturate_ports(METHYL_GENERIC, H_GENERIC)

In [ ]:
portlib.Port.bondable_flavors.insert((0,4))
bonding.saturate_ports(METHYL_GENERIC, OH_GENERIC)

In [ ]:
bonding.saturate_ports(METHYL_GENERIC, H_GENERIC)

### Testing double bonds

In [ ]:
CARBENE = Chem.MolFromSmarts('[2#0:1]=[#6:2]=[#0:3]')
DIOX    = Chem.MolFromSmarts('[#0:1]=[#8:2]')

display(CARBENE)
display(DIOX)

In [ ]:
bonding.saturate_ports(CARBENE, cap=DIOX, flavor_to_saturate=2)

### Testing selectivity between mixed single and double bonds

In [ ]:
C3 = Chem.MolFromSmarts('[#6:1](=[#0:2])(-[#0:3])(-[#0:4])')
C3

In [ ]:
bonding.saturate_ports(C3, cap=DIOX)

### Testing the null case (no bonds formable, but with flavors matching)

# Testing different measures of atom degree

In [ ]:
smiles = 'C'
exp_smiles = specification.expanded_SMILES(smiles)
mol = Chem.MolFromSmiles(exp_smiles)
mol

In [ ]:
Chem.MolFromSmarts(exp_smiles)

In [ ]:
specification.compliant_mol_SMARTS(exp_smiles)

In [ ]:
import pandas as pd
from polymerist.genutils.iteration import iter_len

smarts = ''
mol = Chem.MolFromsmarts(smarts)

records = []
for atom in mol.GetAtoms():
    record = {
        'symbol' : atom.GetSymbol(),
        'map num' : atom.GetAtomMapNum(),
        'num bonds' : iter_len(atom.GetBonds()),
        'impl_valence' : atom.GetImplicitValence(),
        'expl_valence' : atom.GetExplicitValence(),
        'total_valence' : atom.GetTotalValence(),
    }
    records.append(record)
df = pd.DataFrame.from_records(records)
df